# Shingle Dependency Trees for datasketch's Minhash

In [1]:
import sys
sys.path.append("..")

In [2]:
import conllu
import treesimi as ts
import datasketch
import json
import itertools

## Load Dataset

In [3]:
%%capture
!mkdir "../data"
!wget -O "../data/de_hdt-ud-dev.conllu" "https://raw.githubusercontent.com/UniversalDependencies/UD_German-HDT/master/de_hdt-ud-dev.conllu"

In [4]:
dat = conllu.parse(open("../data/de_hdt-ud-dev.conllu").read())
len(dat)

18434

# Generate Shingles and Build LSH
We are trying the following LSH models available in the datasketch package:

1. `MinHashLSH` -- Index is updatable, Threshold is fixed, Num results unknown (Redis support)
2. `MinHashLSHForest` -- Index is updatable, Threshold unknown, Top-N are returned
3. `MinHashLSHEnsemble` -- Index is fixed, Threshold is fixed (Redis support)

In [5]:
%%time

cfg = {
    'use_trunc_leaves': True,
    'use_drop_nodes': False,
    'use_replace_attr': False
}

# Instantiate LSH Forest
lsh1 = datasketch.MinHashLSH(num_perm=256, threshold=0.8)
lsh2 = datasketch.MinHashLSHForest(num_perm=256)
lsh3 = datasketch.MinHashLSHEnsemble(num_perm=256, threshold=0.8, num_part=32)
lsh3examples = []

mhash = []
for i in range(len(dat)):
    # Read tree data 
    adjac = [(t['id'], t['head'], t['deprel']) for t in dat[i]]
    nested = ts.adjac_to_nested_with_attr(adjac)
    nested = ts.remove_node_ids(nested)
    
    # Shingling
    shingled = ts.shingleset(nested, **cfg)
    stringified = [json.dumps(tree).encode('utf-8') for tree in shingled]

    # Create MinHash objects
    m = datasketch.MinHash(num_perm=256)
    for s in stringified:
        m.update(s)

    # Add to LSH
    lsh1.insert(i, m)
    lsh2.add(i, m)
    lsh3examples.append((i, m, len(stringified)))

    # save objects
    mhash.append(m)

# Call index method for LSH Forest
lsh2.index()
# Call index method to build the whole LSH Ensemble
lsh3.index(tuple(lsh3examples))

CPU times: user 2min 19s, sys: 1.09 s, total: 2min 20s
Wall time: 2min 21s


# Query the index

## Just LSH

In [6]:
# try to find any clusters of similar sentences
results = []
for j in range(len(mhash)):
    results.append(lsh1.query(mhash[j]))

In [7]:
tmp = []
for i, res in enumerate(results):
    # number of examples with at least 1 similar example
    if len(res) >= 3 and len(res) <= 10:
        # mininum number of tokens
        if len(dat[i]) >= 8:
            tmp.append(res)

print(len(set(itertools.chain(*tmp))))

32


In [8]:
res = tmp[0]
j = res[0]
print(j)

for i in res:
    print(f"{i:>8d} | {mhash[i].jaccard(mhash[j]):6.4f} | {dat[i].metadata['text']}")

416
     416 | 1.0000 | Das Problem sei mit LetsBuyIt.com bereits geklärt worden .
    5521 | 0.7070 | Im Emissionsprospekt sei diese Aktion nicht erwähnt worden .
    9282 | 0.7070 | Im Januar sollen die Budgetziele neu formuliert werden .


## Top-N Results (LSH Forest)

In [9]:
j = 5000
res = lsh2.query(mhash[j], 5)
print(f"#num results: {len(res)}")

for i in res:
    print(f"{i:>8d} | {mhash[i].jaccard(mhash[j]):6.4f} | {dat[i].metadata['text']}")

#num results: 5
    5000 | 1.0000 | 25 Bilder pro Sekunde in voller PAL-Auflösung lassen sich laut Formac so aufnehmen .
   15371 | 0.1719 | Chip- und Speichertakt liegen dann bei 143 MHz gegenüber rund 105 MHz beim Vorgänger .
    5648 | 0.1172 | 15 Millionen Online-Konten in Deutschland
    9332 | 0.2578 | Neben Siemens wurden weitere sieben Firmen wegen Leistungsmängeln bei Computersystemen mit Geldstrafen belegt .
   11903 | 0.1016 | Gewinn vor Steuern und Gewinn je Anteil sind im Geschäftsjahr 2000 um 70 respektive 73 Prozent gestiegen .


## Containment Query (LSH Ensemble)

In [10]:
j = 5000
res = [key for key in lsh3.query(lsh3examples[j][1], lsh3examples[j][2])]
print(f"#num results: {len(res)}")

for i in res:
    jacc = mhash[i].jaccard(mhash[j])
    if jacc > 0.30:
        print(f"{i:>8d} | {jacc:6.4f} | {dat[i].metadata['text']}")

#num results: 235
   12312 | 0.3008 | Mit solch winzigen Bauelementen könnte man Handys in Zukunft noch kompakter gestalten .
    3841 | 0.3828 | Zusätzliche Rechner lassen sich bei Bedarf jederzeit hinzufügen .
    1485 | 0.3047 | Zudem hatte AOL in Deutschland CompuServe vom zweiten Platz hinter T-Online verdrängt .
    5000 | 1.0000 | 25 Bilder pro Sekunde in voller PAL-Auflösung lassen sich laut Formac so aufnehmen .
